In [3]:
import os
import pandas as pd
from google.cloud import storage

In [ ]:
# # Usar no Google Colab
# from google.colab import auth
# auth.authenticate_user()
# path_folder_silver = '/content/'

In [5]:
# Usar no VSCode
path_folder_silver = os.path.abspath('../data/temp/silver')
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../gcp_key.json"

In [6]:
def extract_year_month(date): # Função para extrair o ano e o mês de uma data
    return date.strftime('%y%m')

year_month = extract_year_month(pd.to_datetime('2017-06-01'))  # Exemplo de uso da função, como se fosse o dia 01 de junho de 2017
print(f'Year-Month: {year_month}')

Year-Month: 1706


In [7]:
def read_csv_cleaned(path_folder_silver, file_name_silver): # Função para ler o arquivo CSV limpo
    df = pd.read_csv(os.path.join(path_folder_silver, file_name_silver))
    
    print(f'Arquivo "{file_name_silver}" lido.')
    return df

file_name_silver = f'mta_{year_month}_cleaned.csv'

df = read_csv_cleaned(path_folder_silver, file_name_silver)
display(df.head(15))
display(df.info())

Arquivo "mta_1706_cleaned.csv" lido.


,RecordedAtTime,DirectionRef,PublishedLineName,VehicleRef,ScheduledArrivalTime,RecordedAtDate,DiffArrivalMins,RecordedTimeRange,ScheduledTimeRange
0,0 days 00:03:49,0,Bx10,NYCT_4223,0 days 00:00:53,2017-06-01,2,0,0
1,1 days 00:03:22,1,Bx1,NYCT_4710,0 days 23:59:38,2017-06-01,3,0,23
2,0 days 00:03:24,0,B31,NYCT_4611,0 days 00:08:00,2017-06-01,-4,0,0
3,0 days 00:03:52,1,B1,NYCT_7141,0 days 00:05:55,2017-06-01,-2,0,0
4,0 days 00:03:41,0,B6,NYCT_7158,0 days 00:05:00,2017-06-01,-1,0,0
5,0 days 00:03:36,0,B4,NYCT_445,0 days 00:01:49,2017-06-01,1,0,0
6,0 days 00:03:53,0,Bx5,NYCT_1280,0 days 00:02:00,2017-06-01,1,0,0
7,1 days 00:03:35,1,M11,NYCT_6372,0 days 23:57:00,2017-06-01,6,0,23
8,1 days 00:03:38,0,Bx32,NYCT_714,0 days 23:59:00,2017-06-01,4,0,23
9,1 days 00:03:46,0,B14,NYCT_7094,0 days 23:49:00,2017-06-01,14,0,23


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1824936 entries, 0 to 1824935
Data columns (total 9 columns):
 #   Column                Dtype 
---  ------                ----- 
 0   RecordedAtTime        object
 1   DirectionRef          int64 
 2   PublishedLineName     object
 3   VehicleRef            object
 4   ScheduledArrivalTime  object
 5   RecordedAtDate        object
 6   DiffArrivalMins       int64 
 7   RecordedTimeRange     int64 
 8   ScheduledTimeRange    int64 
dtypes: int64(4), object(5)
memory usage: 125.3+ MB


None

In [8]:
def change_data_types(df): # Função para alterar os tipos de dados das colunas do DataFrame
    df = df.astype({'PublishedLineName': 'string', 'VehicleRef': 'string'}) # Converte as colunas PublishedLineName e VehicleRef para string
    df['RecordedAtDate'] = pd.to_datetime(df['RecordedAtDate'])  # Converte a coluna RecordedAtDate para datetime
    df['RecordedAtTime'] = pd.to_timedelta(df['RecordedAtTime'])  # Converte a coluna RecordedAtTime para timedelta
    df['ScheduledArrivalTime'] = pd.to_timedelta(df['ScheduledArrivalTime'])  # Converte a coluna ScheduledArrivalTime para timedelta
    
    print("Tipos de dados das colunas alterados.")
    return df

df = change_data_types(df)
display(df.info())

Tipos de dados das colunas alterados.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1824936 entries, 0 to 1824935
Data columns (total 9 columns):
 #   Column                Dtype          
---  ------                -----          
 0   RecordedAtTime        timedelta64[ns]
 1   DirectionRef          int64          
 2   PublishedLineName     string         
 3   VehicleRef            string         
 4   ScheduledArrivalTime  timedelta64[ns]
 5   RecordedAtDate        datetime64[ns] 
 6   DiffArrivalMins       int64          
 7   RecordedTimeRange     int64          
 8   ScheduledTimeRange    int64          
dtypes: datetime64[ns](1), int64(4), string(2), timedelta64[ns](2)
memory usage: 125.3 MB


None

In [ ]:
path_folder_gold_dim = os.path.abspath(f'../data/temp/gold/dim/') # Caminho para a pasta de saída dos arquivos dimensionais

file_name_dim_PublishedLine = f'dim_mta_PublishedLine.csv' # Nome do arquivo para a dimensão PublishedLine
file_name_dim_VehicleRef = f'dim_mta_VehicleRef.csv' # Nome do arquivo para a dimensão VehicleRef

In [ ]:
def read_dim(path_folder_gold_dim, file_name_dim): # Função para ler o arquivo CSV da dimensão PublishedLine já existente
    try:
        df = pd.read_csv(os.path.join(path_folder_gold_dim, file_name_dim))  # Tenta ler o arquivo CSV da dimensão
        print(f'Arquivo "{file_name_dim}" lido.')
        return df
    except:
        print(f'Arquivo "{file_name_dim}" não encontrado.')
        return pd.DataFrame()  # se falhar, cria um DataFrame vazio


df_dim_PublishedLineName = read_dim(path_folder_gold_dim, file_name_dim_PublishedLine)
df_dim_VehicleRef = read_dim(path_folder_gold_dim, file_name_dim_VehicleRef)

Arquivo "dim_mta_PublishedLine.csv" lido.
Arquivo "dim_mta_VehicleRef.csv" lido.


In [ ]:
def update_dim_PublishedLineName(df_new, df_dim): # Função para criar ou atualizar uma Dimensão com a coluna PublishedLineName
    df_new = df_new[['PublishedLineName']].drop_duplicates() # O duplo colchete é necessário para manter o nome da coluna original
    df_dim = pd.concat([df_new, df_dim], ignore_index=True).drop_duplicates()  # Concatena o DataFrame novo com o Dimensão existente
    
    print("Dimensão 'PublishedLineName' criado/atualizado.")
    return df_dim

df_dim_PublishedLineName = update_dim_PublishedLineName(df, df_dim_PublishedLineName)
display(df_dim_PublishedLineName.head(15))
display(df_dim_PublishedLineName.info())

Dimensão 'PublishedLineName' criado/atualizado.


,PublishedLineName
0,Bx10
1,Bx1
2,B31
3,B1
4,B6
5,B4
6,Bx5
7,M11
8,Bx32
9,B14


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 236 entries, 0 to 235
Data columns (total 1 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   PublishedLineName  236 non-null    string
dtypes: string(1)
memory usage: 2.0 KB


None

In [ ]:
def update_dim_VehicleRef(df_new, df_dim): # Função para criar um Dimensão com a coluna VehicleRef
    df_new = df_new[['VehicleRef']].drop_duplicates() # O duplo colchete é necessário para manter o nome da coluna original
    df_dim = pd.concat([df_new, df_dim], ignore_index=True).drop_duplicates()  # Concatena o DataFrame novo com o Dimensão existente
    
    print("Dimensão 'VehicleRef' criado/atualizado.")
    return df_dim

df_dim_VehicleRef = update_dim_VehicleRef(df, df_dim_VehicleRef) # Usa um DataFrame vazio como Dimensão inicial
display(df_dim_VehicleRef.head(15))
display(df_dim_VehicleRef.info())

Dimensão 'VehicleRef' criado/atualizado.


,VehicleRef
0,NYCT_4223
1,NYCT_4710
2,NYCT_4611
3,NYCT_7141
4,NYCT_7158
5,NYCT_445
6,NYCT_1280
7,NYCT_6372
8,NYCT_714
9,NYCT_7094


<class 'pandas.core.frame.DataFrame'>
Index: 4468 entries, 0 to 4467
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   VehicleRef  4468 non-null   string
dtypes: string(1)
memory usage: 69.8 KB


None

In [ ]:
path_folder_gold_fact = os.path.abspath(f'../data/temp/gold/fact/') # Caminho para a pasta de saída dos arquivos factuais

file_name_fact = f'fact_mta_{year_month}.csv' # Nome do arquivo para a tabela factual

In [ ]:
def save_to_csv_gold(path_folder_gold, file_name_gold, df): # Função para salvar o DataFrame em um arquivo CSV na pasta Gold
    os.makedirs(path_folder_gold, exist_ok=True)  # Cria a pasta Gold se ela não existir

    df.to_csv(os.path.join(path_folder_gold, file_name_gold), index=False) # Salva o DataFrame em um arquivo CSV na pasta Gold
    
    folder_type = 'fact' if 'fact' in path_folder_gold else 'dim'  # Determina o tipo de pasta (fact ou dim) com base no caminho do arquivo, para que o arquivo seja enviado para a pasta correta
    
    print(f'Arquivo "{file_name_gold}" salvo na pasta "gold/{folder_type}".')

save_to_csv_gold(path_folder_gold_dim, file_name_dim_PublishedLine, df_dim_PublishedLineName)
save_to_csv_gold(path_folder_gold_dim, file_name_dim_VehicleRef, df_dim_VehicleRef)
save_to_csv_gold(path_folder_gold_fact, file_name_fact, df)

In [ ]:
def upload_to_bucket_gold(path_folder_gold, file_name_gold): # Função para fazer o upload do arquivo CSV para o bucket do GCP
    client = storage.Client() # Cria o cliente para acessar o bucket
    bucket = client.bucket('etl_bus_gps') # Referência pro bucket do GCP
    
    path_folder_file_gold = os.path.join(path_folder_gold, file_name_gold) # Caminho completo do arquivo junto com o nome do arquivo
    
    folder_type = 'fact' if 'fact' in path_folder_gold else 'dim'  # Determina o tipo de pasta (fact ou dim) com base no caminho do arquivo, para que o arquivo seja enviado para a pasta correta no bucket
    
    blob = bucket.blob(f'temp/gold/{folder_type}/{file_name_gold}') # Cria o blob no bucket
    blob.upload_from_filename(path_folder_file_gold) # Faz o upload do arquivo para o bucket
    
    print(f'Arquivo "{file_name_gold}" enviado para a pasta "gold/{folder_type}/" no bucket "{bucket.name}"')

upload_to_bucket_gold(path_folder_gold_dim, file_name_dim_PublishedLine)
upload_to_bucket_gold(path_folder_gold_dim, file_name_dim_VehicleRef) 
upload_to_bucket_gold(path_folder_gold_fact, file_name_fact)

Arquivo "dim_mta_PublishedLine.csv" enviado para a pasta "gold/dim/dim_mta_PublishedLine.csv" no bucket "etl_bus_gps"
Arquivo "dim_mta_VehicleRef.csv" enviado para a pasta "gold/dim/dim_mta_VehicleRef.csv" no bucket "etl_bus_gps"
Arquivo "mta_1706_fact.csv" enviado para a pasta "gold/fact/mta_1706_fact.csv" no bucket "etl_bus_gps"


In [ ]:
def pipeline_gold(year_month): # Função para executar todas as funções de manipulação e upload dos dados
    
    path_folder_silver = os.path.abspath('../data/temp/silver') # Caminho para a pasta de entrada dos arquivos limpos
    file_name_silver = f"mta_{year_month}_cleaned.csv" # Nome do arquivo limpo a ser lido
    
    path_folder_gold_fact = os.path.abspath(f'../data/temp/gold/fact/') # Caminho para a pasta de saída dos arquivos factuais
    file_name_fact = f'fact_mta_{year_month}.csv' # Nome do arquivo para a tabela factual
    
    path_folder_gold_dim = os.path.abspath(f'../data/temp/gold/dim/') # Caminho para a pasta de saída dos arquivos dimensionais
    file_name_dim_PublishedLine = f'dim_mta_PublishedLine.csv' # Nome do arquivo para a dimensão PublishedLine
    file_name_dim_VehicleRef = f'dim_mta_VehicleRef.csv' # Nome do arquivo para a dimensão VehicleRef
    
    print('Iniciando o pipeline Gold...')
    df = read_csv_cleaned(path_folder_silver, file_name_silver) # Lê o arquivo CSV limpo
    df = change_data_types(df) # Altera os tipos de dados das colunas do DataFrame
    
    df_dim_PublishedLineName = read_dim(path_folder_gold_dim, file_name_dim_PublishedLine) # Lê o arquivo CSV da dimensão PublishedLine já existente
    df_dim_PublishedLineName = update_dim_PublishedLineName(df, df_dim_PublishedLineName) # Atualiza ou cria a Dimensão PublishedLineName
    df_dim_VehicleRef = read_dim(path_folder_gold_dim, file_name_dim_VehicleRef) # Lê o arquivo CSV da dimensão VehicleRef já existente
    df_dim_VehicleRef = update_dim_VehicleRef(df, df_dim_VehicleRef) # Atualiza ou cria a Dimensão VehicleRef
    
    data_to_save = { # Dicionário para armazenar: caminhos da pasta de saída, nomes dos arquivos e DataFrames a serem salvos
        path_folder_gold_dim: {file_name_dim_PublishedLine: df_dim_PublishedLineName, file_name_dim_VehicleRef: df_dim_VehicleRef},
        path_folder_gold_fact: {file_name_fact: df}
    }
    
    for path_folder_gold, files_names_gold in data_to_save.items(): # Itera sobre os caminhos da pasta de saída, nomes dos arquivos e DataFrames a serem salvos
        for file_name_gold, df in files_names_gold.items():
            save_to_csv_gold(path_folder_gold, file_name_gold, df) # Salva o DataFrame em um arquivo CSV na pasta Gold
            upload_to_bucket_gold(path_folder_gold, file_name_gold) # Faz o upload do arquivo CSV para o bucket do GCP
    
    print(f'Pipeline Gold concluído para o arquivo "{file_name_silver}"!!\n')
    
pipeline_gold(year_month)  # Executa o pipeline Gold

Iniciando o pipeline Gold...
Arquivo "mta_1706_cleaned.csv" lido.
Tipos de dados das colunas alterados.
Lendo arquivo "dim_mta_PublishedLine.csv"...
Dimensão 'PublishedLineName' criado/atualizado.
Lendo arquivo "dim_mta_VehicleRef.csv"...
Dimensão 'VehicleRef' criado/atualizado.
Arquivo "dim_mta_PublishedLine.csv" salvo na pasta "gold/dim".
Arquivo "dim_mta_PublishedLine.csv" enviado para a pasta "gold/dim/dim_mta_PublishedLine.csv" no bucket "etl_bus_gps"
Arquivo "dim_mta_VehicleRef.csv" salvo na pasta "gold/dim".
Arquivo "dim_mta_VehicleRef.csv" enviado para a pasta "gold/dim/dim_mta_VehicleRef.csv" no bucket "etl_bus_gps"
Arquivo "fact_mta_1706.csv" salvo na pasta "gold/fact".
Arquivo "fact_mta_1706.csv" enviado para a pasta "gold/fact/fact_mta_1706.csv" no bucket "etl_bus_gps"
Pipeline Gold concluído para o arquivo "mta_1706_cleaned.csv"!!



In [48]:
year_month_list = [] # Lista para armazenar os anos e meses

year_month_list.append(extract_year_month(pd.to_datetime('2017-06-01'))) # Adiciona o ano e mês de junho de 2017
year_month_list.append(extract_year_month(pd.to_datetime('2017-08-01'))) # Adiciona o ano e mês de agosto de 2017
year_month_list.append(extract_year_month(pd.to_datetime('2017-10-01'))) # Adiciona o ano e mês de outubro de 2017
year_month_list.append(extract_year_month(pd.to_datetime('2017-12-01'))) # Adiciona o ano e mês de dezembro de 2017

for year_month in year_month_list: # Itera sobre os anos e meses na lista, simulando o Airflow
    pipeline_gold(year_month)  # Executa o pipeline para o ano e mês especificados

Iniciando o pipeline Gold...
Arquivo "mta_1706_cleaned.csv" lido.
Tipos de dados das colunas alterados.
Lendo arquivo "dim_mta_PublishedLine.csv"...
Arquivo "dim_mta_PublishedLine.csv" não encontrado.
Dimensão 'PublishedLineName' criado/atualizado.
Lendo arquivo "dim_mta_VehicleRef.csv"...
Arquivo "dim_mta_VehicleRef.csv" não encontrado.
Dimensão 'VehicleRef' criado/atualizado.
Arquivo "dim_mta_PublishedLine.csv" salvo na pasta "gold/dim".
Arquivo "dim_mta_PublishedLine.csv" enviado para a pasta "gold/dim/dim_mta_PublishedLine.csv" no bucket "etl_bus_gps"
Arquivo "dim_mta_VehicleRef.csv" salvo na pasta "gold/dim".
Arquivo "dim_mta_VehicleRef.csv" enviado para a pasta "gold/dim/dim_mta_VehicleRef.csv" no bucket "etl_bus_gps"
Arquivo "fact_mta_1706.csv" salvo na pasta "gold/fact".
Arquivo "fact_mta_1706.csv" enviado para a pasta "gold/fact/fact_mta_1706.csv" no bucket "etl_bus_gps"
Pipeline Gold concluído para o arquivo "mta_1706_cleaned.csv"!!

Iniciando o pipeline Gold...
Arquivo "mta_